### Konstantina Andronikou 

## Comparison of Names as a Robustness test

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
#removing warnings 
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Maria Smith likes to eat pizza")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'likes',
   'description': '[ARG0: Maria Smith] [V: likes] [ARG1: to eat pizza]',
   'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']},
  {'verb': 'eat',
   'description': '[ARG0: Maria Smith] likes to [V: eat] [ARG1: pizza]',
   'tags': ['B-ARG0', 'I-ARG0', 'O', 'O', 'B-V', 'B-ARG1']}],
 'words': ['Maria', 'Smith', 'likes', 'to', 'eat', 'pizza']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Maria Smith likes to eat pizza"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'likes',
     'description': '[ARG0: Maria Smith] [V: likes] [ARG1: to eat pizza]',
     'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']},
    {'verb': 'eat',
     'description': '[ARG0: Maria Smith] likes to [V: eat] [ARG1: pizza]',
     'tags': ['B-ARG0', 'I-ARG0', 'O', 'O', 'B-V', 'B-ARG1']}],
   'words': ['Maria', 'Smith', 'likes', 'to', 'eat', 'pizza']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name'],meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:

editor = Editor()

# create examples
t = editor.template('{first_name} {last_name} likes to eat pizza', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'last_name': 'Sullivan', 'first_name': 'Harold'}, {'last_name': 'Bennett', 'first_name': 'Martin'}, {'last_name': 'Brown', 'first_name': 'Roy'}, {'last_name': 'Gordon', 'first_name': 'Joe'}, {'last_name': 'Coleman', 'first_name': 'Florence'}, {'last_name': 'Lewis', 'first_name': 'Alan'}, {'last_name': 'Stevens', 'first_name': 'Fred'}, {'last_name': 'Howard', 'first_name': 'Karen'}, {'last_name': 'King', 'first_name': 'Sarah'}, {'last_name': 'Gray', 'first_name': 'Charles'}, {'last_name': 'Reynolds', 'first_name': 'Amanda'}, {'last_name': 'Martin', 'first_name': 'Gary'}, {'last_name': 'Davies', 'first_name': 'Matt'}, {'last_name': 'Baker', 'first_name': 'Carolyn'}, {'last_name': 'Crawford', 'first_name': 'Jean'}, {'last_name': 'Wood', 'first_name': 'Ian'}, {'last_name': 'Howard', 'first_name': 'Sarah'}, {'last_name': 'Butler', 'first_name': 'Deborah'}, {'last_name': 'Jackson', 'first_name': 'Dick'}, {'last_name': 'Price', 'first_name': 'Bobby'}, {'last_name': 'Adams', 'first_name

In [14]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Western_names_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Harold', 'Sullivan', 'likes', 'to', 'eat', 'pizza'] [ True]
['Martin', 'Bennett', 'likes', 'to', 'eat', 'pizza'] [ True]
['Roy', 'Brown', 'likes', 'to', 'eat', 'pizza'] [ True]
['Joe', 'Gordon', 'likes', 'to', 'eat', 'pizza'] [ True]
['Florence', 'Coleman', 'likes', 'to', 'eat', 'pizza'] [ True]
['Alan', 'Lewis', 'likes', 'to', 'eat', 'pizza'] [ True]
['Fred', 'Stevens', 'likes', 'to', 'eat', 'pizza'] [ True]
['Karen', 'Howard', 'likes', 'to', 'eat', 'pizza'] [ True]
['Sarah', 'King', 'likes', 'to', 'eat', 'pizza'] [ True]
['Charles', 'Gray', 'likes', 'to', 'eat', 'pizza'] [ True]
['Amanda', 'Reynolds', 'likes', 'to', 'eat', 'pizza'] [ True]
['Gary', 'Martin', 'likes', 'to', 'eat', 'pizza'] [ True]
['Matt', 'Davies', 'likes', 'to', 'eat', 'pizza'] [ True]
['Carolyn', 'Baker', 'likes', 'to', 'eat', 'pizza'] [ True]
['Jean', 'Crawford', 'likes', 'to', 'eat', 'pizza'] [ True]
['Ian', 'Wood', 'likes', 'to', 'eat', '

In [15]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Western_names_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Non-Western Names 

In [16]:
first = [x.split()[0] for x in editor.lexicons.male_from.Turkey +  editor.lexicons.female_from.Turkey]
last = [x.split()[0] for x in editor.lexicons.last_from.Turkey]

In [18]:
t = editor.template("{first_name} {last_name} likes to eat pizza.", first_name=first, last_name=last, meta=True, nsamples=1000)
test = MFT(**t, expect=expect_arg0)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
with open ('Output/Non-Western_names_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    205 (20.5%)

Example fails:
[ARG0: Gökhan] Şahin [V: likes] [ARG1: to eat pizza] .
----
[ARG0: Güler] Subaşı [V: likes] [ARG1: to eat pizza] .
----
[ARG0: İlayda] Akgün [V: likes] [ARG1: to eat pizza] .
----
['Harold', 'Sullivan', 'likes', 'to', 'eat', 'pizza'] [ True]
['Martin', 'Bennett', 'likes', 'to', 'eat', 'pizza'] [ True]
['Roy', 'Brown', 'likes', 'to', 'eat', 'pizza'] [ True]
['Joe', 'Gordon', 'likes', 'to', 'eat', 'pizza'] [ True]
['Florence', 'Coleman', 'likes', 'to', 'eat', 'pizza'] [ True]
['Alan', 'Lewis', 'likes', 'to', 'eat', 'pizza'] [ True]
['Fred', 'Stevens', 'likes', 'to', 'eat', 'pizza'] [ True]
['Karen', 'Howard', 'likes', 'to', 'eat', 'pizza'] [ True]
['Sarah', 'King', 'likes', 'to', 'eat', 'pizza'] [ True]
['Charles', 'Gray', 'likes', 'to', 'eat', 'pizza'] [ True]
['Amanda', 'Reynolds', 'likes', 'to', 'eat', 'pizza'] [ True]
['Gary', 'Martin', 'likes', 'to', 'eat', 'pizza'] [ True]
['Matt', 'Davies',

In [19]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Non-Western_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 